In [2]:
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Input, Dense, Conv1D, GRU, Dropout, Flatten, Activation
from keras.layers import concatenate, add, Lambda
from keras.models import Model, Sequential
import keras.backend as K

from esgflib.data.datasets import get_melbourne_data, split_train_test_data, create_test_data
from matplotlib import pyplot as plt
from esgflib.features.features import create_training_points

import pickle
import datetime


In [3]:
#LSTNet model with GRU gates

def make_model(hidC = 30,Ck = 3,hidR = 20,hidS = 30,skip = 15,loss = "mse",pt =3):

    x = Input(shape=(history_days, 1))

    # CNN
    c = Conv1D(hidC, Ck, activation='relu', use_bias = True, kernel_regularizer = keras.regularizers.L1(0.005))(x)

    # RNN

    r = GRU(hidR, kernel_regularizer = keras.regularizers.L1(0.005))(c)
    r = Lambda(lambda k: K.reshape(k, (-1, hidR)))(r)

    # skip-RNN
    if skip > 0:
        # Reshape the data to easily setup the skip-RNN
        s = Lambda(lambda k: k[:, int(-pt*skip):, :])(c)
        s = Lambda(lambda k: K.reshape(k, (-1, pt, skip, hidC)))(s)
        s = Lambda(lambda k: K.permute_dimensions(k, (0,2,1,3)))(s)
        s = Lambda(lambda k: K.reshape(k, (-1, pt, hidC)))(s)

        s = GRU(hidS, kernel_regularizer = keras.regularizers.L1(0.005))(s)
        s = Lambda(lambda k: K.reshape(k, (-1, skip*hidS)))(s)
        r = concatenate([r,s])

    res = Dense(horizon_days, kernel_regularizer = keras.regularizers.L1(0.005))(r)
 


       

    res = Activation("relu")(res)
    model = Model(inputs=x, outputs=res)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss, metrics = [keras.metrics.mse])
    return model

In [8]:

# Get the data
melbourne_data = get_melbourne_data()


test_years = [1987,1988,1989]
test_months = [3,6, 12]

eval = {}

for year in test_years:
    eval[year] = []
    for month in test_months:
        print("Year:", year, "prediction of the next", month, "months")
        # Will evaluate the model on the next 30*month 
        horizon_days = month *30
        #Models will be trained on 2 times the history to make sure to encounter the patterns to predicts
        history_days = horizon_days * 2
        
        
        # Get data
        train_data, _ = split_train_test_data(melbourne_data, split_year=year)
        X_test, Y_test = create_test_data(melbourne_data, history_days=history_days, horizon_days=horizon_days, year = year)
        # Index
        train_data.set_index('Date', inplace=True)

        # Create test points

        X_train, Y_train = create_training_points(train_data, history_days=history_days, horizon_days=horizon_days)
        
        
        #Configuration of the model
        
        model = make_model(hidC = 30,Ck = 3,hidR = 20,hidS = 30,skip = 15,loss = "mse",pt =int((history_days-3)/15))
        model.fit(X_train, Y_train, epochs = horizon_days * 10, batch_size = X_train.shape[0], verbose = False)
        eval[year].append(model.evaluate(X_test, Y_test)[1])
        with open("model"+str(year)+"."+str(month)+"."+str(int(eval[year][-1]))+".pkl", "wb") as f:
            pickle.dump(model, f)
with open("eval.pkl", "wb") as f:
    pickle.dump(eval, f)

Year: 1987 prediction of the next 3 months
1/1 [==============================] - 1s 608ms/step - loss: 12.1762 - mean_squared_error: 5.6959


INFO:tensorflow:Assets written to: ram://85293dd8-ec02-4513-a63f-66c74969d038/assets


INFO:tensorflow:Assets written to: ram://85293dd8-ec02-4513-a63f-66c74969d038/assets


Year: 1987 prediction of the next 6 months
1/1 [==============================] - 1s 591ms/step - loss: 17.5479 - mean_squared_error: 6.6927


INFO:tensorflow:Assets written to: ram://adf20470-4428-48b5-a602-fb1e52ea6b12/assets


...

INFO:tensorflow:Assets written to: ram://2a1f6006-f334-483c-a9f2-afd8945f4e97/assets


INFO:tensorflow:Assets written to: ram://2a1f6006-f334-483c-a9f2-afd8945f4e97/assets


Year: 1989 prediction of the next 12 months
1/1 [==============================] - 1s 562ms/step - loss: 30.4211 - mean_squared_error: 15.3725


INFO:tensorflow:Assets written to: ram://4b0834b9-0d4a-4a53-b0e5-719e03a7e984/assets


INFO:tensorflow:Assets written to: ram://4b0834b9-0d4a-4a53-b0e5-719e03a7e984/assets


In [9]:
with open("eval.pkl", "rb") as f:
    eval = pickle.load(f)
print(eval)

{1987: [5.695873260498047, 6.692675590515137, 9.908455848693848], 1988: [9.479012489318848, 10.612396240234375, 13.302288055419922], 1989: [13.896384239196777, 13.319490432739258, 15.37254810333252]}
